# Lab 2.4.5: Architecture Comparison

**Module:** 2.4 - Efficient Architectures  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this lab, you will:
- [ ] Systematically compare Mamba, Transformer, and MoE architectures
- [ ] Benchmark perplexity, speed, and memory usage
- [ ] Understand when to use each architecture
- [ ] Create a decision framework for architecture selection

---

## 📚 Prerequisites

- Completed: Labs 2.4.1-2.4.4
- Knowledge of: Perplexity, benchmarking methodology
- Hardware: DGX Spark (128GB) recommended for full comparison

---

## 🌍 Real-World Context

**Choosing the Right Architecture**

In production, you'll face decisions like:
- "We need to process 100K-token documents" → Mamba?
- "We want the best quality regardless of cost" → Dense Transformer?
- "We need 70B-level quality at 7B compute cost" → MoE?
- "We need both quality AND long context" → Hybrid (Jamba)?

This lab gives you the data to make these decisions.

---

## Part 1: Setup and Model Loading

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import gc
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from collections import defaultdict

from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"Memory: {total_memory:.1f} GB")

def clear_memory():
    """Clear GPU memory between models."""
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

def get_memory_usage():
    """Get current GPU memory usage in GB."""
    return torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0

In [ ]:
@dataclass
class BenchmarkResult:
    """Container for benchmark results."""
    model_name: str
    architecture: str
    total_params_b: float
    active_params_b: float
    memory_gb: float
    perplexity: Optional[float]
    tokens_per_second: Dict[int, float]  # context_length -> speed
    peak_memory: Dict[int, float]  # context_length -> memory

# Define models to compare
# Adjust based on your available memory
MODELS_TO_COMPARE = {
    "mamba-2.8b": {
        "name": "state-spaces/mamba-2.8b-hf",
        "architecture": "Mamba",  # State Space Models
        "active_ratio": 1.0,  # All params active
    },
    # Smaller transformer for fair comparison
    "phi-2": {
        "name": "microsoft/phi-2",
        "architecture": "Transformer",
        "active_ratio": 1.0,
    },
    # MoE model (if memory allows)
    # "deepseek-moe-16b": {
    #     "name": "deepseek-ai/deepseek-moe-16b-base",
    #     "architecture": "MoE",
    #     "active_ratio": 0.156,  # 2.5B / 16B
    # },
}

print(f"Will compare {len(MODELS_TO_COMPARE)} models:")
for name, config in MODELS_TO_COMPARE.items():
    print(f"  - {name} ({config['architecture']})")

---

## Part 2: Benchmarking Framework

In [ ]:
class ArchitectureBenchmark:
    """
    Comprehensive benchmarking suite for architecture comparison.
    """
    
    def __init__(self, context_lengths: List[int] = None):
        self.context_lengths = context_lengths or [512, 1024, 2048, 4096, 8192]
        self.results: List[BenchmarkResult] = []
    
    def load_model(self, model_name: str) -> Tuple:
        """Load a model and tokenizer."""
        clear_memory()
        
        print(f"Loading {model_name}...")
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True,
        )
        
        return model, tokenizer
    
    def benchmark_speed(self, model, tokenizer, context_lengths: List[int],
                       generation_length: int = 50,
                       warmup: int = 2, runs: int = 3) -> Dict[int, float]:
        """
        Benchmark generation speed at various context lengths.
        """
        speeds = {}
        peak_memories = {}
        
        for ctx_len in context_lengths:
            print(f"  Testing context: {ctx_len}...")
            
            try:
                # Create input
                input_ids = torch.randint(
                    100, tokenizer.vocab_size - 100,
                    (1, ctx_len), device=device
                )
                
                # Warmup
                for _ in range(warmup):
                    with torch.no_grad():
                        _ = model.generate(
                            input_ids[:, :min(512, ctx_len)],
                            max_new_tokens=5,
                            do_sample=False,
                            pad_token_id=tokenizer.pad_token_id,
                        )
                
                # Clear and benchmark
                torch.cuda.empty_cache()
                torch.cuda.reset_peak_memory_stats()
                
                times = []
                for _ in range(runs):
                    torch.cuda.synchronize()
                    start = time.perf_counter()
                    
                    with torch.no_grad():
                        _ = model.generate(
                            input_ids,
                            max_new_tokens=generation_length,
                            do_sample=False,
                            pad_token_id=tokenizer.pad_token_id,
                        )
                    
                    torch.cuda.synchronize()
                    times.append(time.perf_counter() - start)
                
                avg_time = np.mean(times)
                speeds[ctx_len] = generation_length / avg_time
                peak_memories[ctx_len] = torch.cuda.max_memory_allocated() / 1e9
                
                print(f"    {speeds[ctx_len]:.1f} tok/s, {peak_memories[ctx_len]:.1f} GB peak")
                
            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    print(f"    OOM at {ctx_len}")
                    speeds[ctx_len] = 0
                    peak_memories[ctx_len] = float('inf')
                    torch.cuda.empty_cache()
                else:
                    raise
        
        return speeds, peak_memories
    
    def benchmark_perplexity(self, model, tokenizer, 
                            text: str = None, max_length: int = 2048) -> float:
        """
        Calculate perplexity on sample text.
        """
        if text is None:
            # Use a standard test text
            text = """The Transformer architecture has revolutionized natural language processing. 
            Introduced in the paper "Attention is All You Need", it relies on self-attention 
            mechanisms to process sequences in parallel. However, the quadratic complexity of 
            attention poses challenges for long sequences. Recent innovations like Mamba and 
            Mixture of Experts offer alternative approaches with different trade-offs."""
            text = text * 20  # Repeat for longer sequence
        
        encodings = tokenizer(text, return_tensors="pt", truncation=True, 
                             max_length=max_length)
        input_ids = encodings.input_ids.to(device)
        
        with torch.no_grad():
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss
        
        perplexity = torch.exp(loss).item()
        return perplexity
    
    def run_benchmark(self, model_configs: Dict) -> List[BenchmarkResult]:
        """
        Run complete benchmark on all models.
        """
        results = []
        
        for model_key, config in model_configs.items():
            print(f"\n{'='*60}")
            print(f"Benchmarking: {model_key}")
            print(f"{'='*60}")
            
            try:
                model, tokenizer = self.load_model(config['name'])
                
                # Get model info
                total_params = sum(p.numel() for p in model.parameters())
                active_params = total_params * config['active_ratio']
                memory = get_memory_usage()
                
                print(f"Loaded: {total_params/1e9:.2f}B params, {memory:.1f} GB")
                
                # Speed benchmark
                print("\nSpeed benchmark:")
                speeds, peak_memories = self.benchmark_speed(
                    model, tokenizer, self.context_lengths
                )
                
                # Perplexity benchmark
                print("\nPerplexity benchmark:")
                try:
                    ppl = self.benchmark_perplexity(model, tokenizer)
                    print(f"  Perplexity: {ppl:.2f}")
                except Exception as e:
                    print(f"  Perplexity failed: {e}")
                    ppl = None
                
                result = BenchmarkResult(
                    model_name=model_key,
                    architecture=config['architecture'],
                    total_params_b=total_params / 1e9,
                    active_params_b=active_params / 1e9,
                    memory_gb=memory,
                    perplexity=ppl,
                    tokens_per_second=speeds,
                    peak_memory=peak_memories,
                )
                results.append(result)
                
                # Cleanup
                del model, tokenizer
                clear_memory()
                
            except Exception as e:
                print(f"Failed to benchmark {model_key}: {e}")
                clear_memory()
        
        self.results = results
        return results

print("Benchmark framework ready!")

In [ ]:
# Run benchmarks
benchmark = ArchitectureBenchmark(
    context_lengths=[512, 1024, 2048, 4096]  # Adjust based on memory
)

results = benchmark.run_benchmark(MODELS_TO_COMPARE)
print(f"\n✅ Completed {len(results)} benchmarks!")

---

## Part 3: Visualizing Results

In [ ]:
def visualize_benchmark_results(results: List[BenchmarkResult]):
    """
    Create comprehensive visualizations of benchmark results.
    """
    if not results:
        print("No results to visualize")
        return
    
    # Get common context lengths
    all_contexts = set()
    for r in results:
        all_contexts.update(r.tokens_per_second.keys())
    contexts = sorted(all_contexts)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Architecture colors: Mamba (State Space Models), Transformer, MoE
    colors = {'Mamba': '#27AE60', 'Transformer': '#E74C3C', 'MoE': '#3498DB'}
    
    # 1. Speed comparison
    ax = axes[0, 0]
    for result in results:
        ctx = sorted(result.tokens_per_second.keys())
        speeds = [result.tokens_per_second[c] for c in ctx]
        color = colors.get(result.architecture, '#9B59B6')
        ax.plot(ctx, speeds, 'o-', label=result.model_name, 
               linewidth=2, markersize=8, color=color)
    
    ax.set_xlabel('Context Length (tokens)')
    ax.set_ylabel('Tokens per Second')
    ax.set_title('Generation Speed vs Context Length', fontweight='bold')
    ax.legend()
    ax.set_xscale('log', base=2)
    ax.grid(True, alpha=0.3)
    
    # 2. Memory comparison
    ax = axes[0, 1]
    for result in results:
        ctx = sorted(result.peak_memory.keys())
        mems = [result.peak_memory[c] if result.peak_memory[c] < float('inf') else np.nan 
                for c in ctx]
        color = colors.get(result.architecture, '#9B59B6')
        ax.plot(ctx, mems, 's-', label=result.model_name,
               linewidth=2, markersize=8, color=color)
    
    ax.axhline(y=128, color='gray', linestyle='--', label='DGX Spark (128GB)')
    ax.set_xlabel('Context Length (tokens)')
    ax.set_ylabel('Peak Memory (GB)')
    ax.set_title('Memory Usage vs Context Length', fontweight='bold')
    ax.legend()
    ax.set_xscale('log', base=2)
    ax.grid(True, alpha=0.3)
    
    # 3. Model size comparison
    ax = axes[1, 0]
    model_names = [r.model_name for r in results]
    total_params = [r.total_params_b for r in results]
    active_params = [r.active_params_b for r in results]
    
    x = np.arange(len(model_names))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, total_params, width, label='Total Params', 
                  color='#3498DB')
    bars2 = ax.bar(x + width/2, active_params, width, label='Active Params',
                  color='#27AE60')
    
    ax.set_xlabel('Model')
    ax.set_ylabel('Parameters (Billions)')
    ax.set_title('Total vs Active Parameters', fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(model_names, rotation=15)
    ax.legend()
    
    # 4. Perplexity comparison
    ax = axes[1, 1]
    ppls = [r.perplexity if r.perplexity else 0 for r in results]
    bars_colors = [colors.get(r.architecture, '#9B59B6') for r in results]
    
    bars = ax.bar(model_names, ppls, color=bars_colors)
    ax.set_xlabel('Model')
    ax.set_ylabel('Perplexity (lower = better)')
    ax.set_title('Model Perplexity Comparison', fontweight='bold')
    ax.set_xticklabels(model_names, rotation=15)
    
    # Add legend for architecture colors
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=c, label=a) for a, c in colors.items()]
    ax.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.show()

visualize_benchmark_results(results)

In [ ]:
# Create summary table
def create_summary_table(results: List[BenchmarkResult]) -> pd.DataFrame:
    """
    Create a summary comparison table.
    """
    data = []
    
    for r in results:
        # Get speed at common context length (1024)
        speed_1k = r.tokens_per_second.get(1024, r.tokens_per_second.get(512, 0))
        speed_4k = r.tokens_per_second.get(4096, 0)
        
        data.append({
            'Model': r.model_name,
            'Architecture': r.architecture,
            'Total Params (B)': f"{r.total_params_b:.2f}",
            'Active Params (B)': f"{r.active_params_b:.2f}",
            'Model Memory (GB)': f"{r.memory_gb:.1f}",
            'Speed @1K (tok/s)': f"{speed_1k:.1f}" if speed_1k else "N/A",
            'Speed @4K (tok/s)': f"{speed_4k:.1f}" if speed_4k else "N/A",
            'Perplexity': f"{r.perplexity:.2f}" if r.perplexity else "N/A",
        })
    
    df = pd.DataFrame(data)
    return df

if results:
    summary_df = create_summary_table(results)
    print("\n📊 ARCHITECTURE COMPARISON SUMMARY")
    print("=" * 100)
    print(summary_df.to_string(index=False))

---

## Part 4: Decision Framework

In [ ]:
def recommend_architecture(requirements: Dict) -> str:
    """
    Recommend an architecture based on requirements.
    
    Args:
        requirements: Dict with keys like:
            - max_context_length: int
            - quality_priority: 'high', 'medium', 'low'
            - memory_limit_gb: float
            - speed_priority: 'high', 'medium', 'low'
            - use_case: 'chat', 'document', 'code', 'general'
    
    Returns:
        Recommendation string with reasoning
    """
    ctx_len = requirements.get('max_context_length', 4096)
    quality = requirements.get('quality_priority', 'medium')
    memory = requirements.get('memory_limit_gb', 24)
    speed = requirements.get('speed_priority', 'medium')
    use_case = requirements.get('use_case', 'general')
    
    recommendations = []
    
    # Long context requirement
    if ctx_len > 32000:
        recommendations.append({
            'arch': 'Mamba',
            'score': 10,
            'reason': 'Linear memory scaling essential for 32K+ context'
        })
        recommendations.append({
            'arch': 'Jamba (Hybrid)',
            'score': 9,
            'reason': 'Combines Mamba efficiency with Attention quality'
        })
    
    # Quality priority
    if quality == 'high':
        recommendations.append({
            'arch': 'Dense Transformer',
            'score': 8 if ctx_len < 16000 else 5,
            'reason': 'Best quality for most tasks (if context fits)'
        })
    
    # Memory constraint with large model need
    if memory < 50 and requirements.get('model_size', 'large') == 'large':
        recommendations.append({
            'arch': 'MoE',
            'score': 9,
            'reason': 'More parameters with less compute/memory'
        })
    
    # Speed priority
    if speed == 'high':
        recommendations.append({
            'arch': 'Mamba',
            'score': 8,
            'reason': 'No KV cache overhead, consistent speed'
        })
    
    # Use case specific
    if use_case == 'document':
        recommendations.append({
            'arch': 'Mamba',
            'score': 9,
            'reason': 'Document processing benefits from linear scaling'
        })
    elif use_case == 'code':
        recommendations.append({
            'arch': 'Dense Transformer',
            'score': 8,
            'reason': 'Code completion benefits from precise attention'
        })
    
    # Sort by score
    recommendations.sort(key=lambda x: -x['score'])
    
    # Build recommendation text
    text = "\n🎯 ARCHITECTURE RECOMMENDATION\n" + "=" * 50 + "\n"
    text += f"\nRequirements:\n"
    for k, v in requirements.items():
        text += f"  - {k}: {v}\n"
    
    text += f"\nTop Recommendations:\n"
    for i, rec in enumerate(recommendations[:3], 1):
        text += f"\n  {i}. {rec['arch']} (Score: {rec['score']}/10)\n"
        text += f"     Reason: {rec['reason']}\n"
    
    return text

# Example recommendations
print(recommend_architecture({
    'max_context_length': 100000,
    'quality_priority': 'high',
    'memory_limit_gb': 128,
    'use_case': 'document'
}))

print(recommend_architecture({
    'max_context_length': 4096,
    'quality_priority': 'high',
    'memory_limit_gb': 24,
    'use_case': 'code'
}))

---

## Part 5: Architecture Selection Flowchart

In [ ]:
# Visual decision tree
decision_tree = """
🔄 ARCHITECTURE SELECTION FLOWCHART
══════════════════════════════════════════════════════════════════════

                    ┌─────────────────────┐
                    │ Need context > 32K? │
                    └──────────┬──────────┘
                               │
              ┌────────────────┴────────────────┐
              │ YES                              │ NO
              ▼                                  ▼
    ┌──────────────────┐              ┌──────────────────┐
    │ Need best quality│              │ Memory limited?  │
    │ at long context? │              │ (<50GB for 70B+) │
    └────────┬─────────┘              └────────┬─────────┘
             │                                  │
    ┌────────┴────────┐              ┌─────────┴─────────┐
    │ YES        │ NO │              │ YES          │ NO │
    ▼            ▼    │              ▼              ▼    │
╔═══════════╗ ╔═══════╗         ╔════════════╗ ╔════════════════╗
║  JAMBA    ║ ║ MAMBA ║         ║    MoE     ║ ║ Dense          ║
║ (Hybrid)  ║ ║       ║         ║ (Sparse)   ║ ║ Transformer    ║
╚═══════════╝ ╚═══════╝         ╚════════════╝ ╚════════════════╝
 Best of both   Fastest          More params,   Best quality
 worlds         for long ctx     less compute   (if fits!)

══════════════════════════════════════════════════════════════════════

📋 QUICK REFERENCE:

┌─────────────────┬──────────────┬──────────────┬─────────────────┐
│ Use Case        │ Recommended  │ Context Limit│ Notes           │
├─────────────────┼──────────────┼──────────────┼─────────────────┤
│ Chatbot         │ Transformer  │ ~8K tokens   │ Quality matters │
│ Document QA     │ Mamba/Jamba  │ 100K+ tokens │ Long context    │
│ Code Generation │ Transformer  │ ~16K tokens  │ Precision needed│
│ Audio/Video     │ Mamba        │ 1M+ samples  │ Streaming       │
│ Cost-Efficient  │ MoE          │ ~8K tokens   │ 10x efficiency  │
│ Edge Deployment │ Small Trans. │ ~4K tokens   │ Memory limited  │
└─────────────────┴──────────────┴──────────────┴─────────────────┘
"""

print(decision_tree)

---

## 🎉 Checkpoint

You've learned:
- ✅ How to systematically benchmark different architectures
- ✅ Speed, memory, and quality tradeoffs
- ✅ When to choose each architecture type
- ✅ A decision framework for real-world selection

---

## ✋ Try It Yourself

Add another model to the comparison (e.g., Qwen-MoE or a different transformer size) and re-run the benchmarks.

In [ ]:
# Your code here



---

## 🧹 Cleanup

In [ ]:
clear_memory()
print("✅ Cleanup complete!")